In [ ]:
!pip install geemap

In [ ]:
!pip install earthengine-api

In [ ]:
import ee
import geemap

#### This code removes the proxy settings from your environment, which can be helpful if you're having connectivity issues due to a proxy configuration.

In [ ]:
#import os
#os.environ['http_proxy'] = '' #Removes the environment variable that defines the proxy for HTTP connections.
#os.environ['https_proxy'] = '' #Removes the environment variable that defines the proxy for HTTP connections.

In [ ]:
ee.Authenticate()
ee.Initialize(project='my-project')

In [ ]:
geemap.ee_initialize()

This Python snippet uses IPython.display to show an HTML link in a Jupyter Notebook. The link opens geojson.io centered at specific coordinates, allowing users to interactively draw a geographic shape (e.g., a rectangle) on the map.

In [ ]:
from IPython.display import display, HTML

# Create the URL for GeoJSON.io so you can draw the rectangle
url = "https://geojson.io/#map=7.16/42.741/-7.906"

# Create a link to open GeoJSON.io
display(HTML(f'<a href="{url}" target="_blank">Click here to open GeoJSON.io and draw a rectangle.</a>'))

This script analyzes wildfire impact using Sentinel-2 imagery by calculating the Normalized Burn Ratio (NBR) for pre- and post-fire periods. It defines a polygon region of interest, filters images with low cloud cover for both timeframes, computes NBR values, and displays the results in a split-panel map for visual comparison. A legend and text annotations are added to help interpret the levels of vegetation damage.

In [ ]:
# Coordinates extracted from GeoJSON.io
# Make sure to copy the coordinates from the GeoJSON generated when you draw the rectangle on GeoJSON.io.
coordinates = [
    [
        -8.62202690794848,
              42.3938610820762],
        [-8.62202690794848,
              42.14269095261844],
        [-7.986988260972595,
              42.1426909526184],
        [-7.986988260972595,
              42.3938610820762],
        [ -8.62202690794848,
              42.3938610820762]
    ]

# Create a geometry from the coordinates
geometry = ee.Geometry.Polygon(coordinates)

# Function to calculate NBR (Normalized Burn Ratio)
def calculate_nbr(image):
    return image.normalizedDifference(['B8A', 'B12']).rename('NBR')

# Pre-Fire Image Collection
img_pre_collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterDate('2017-10-01', '2017-10-10') \
    .select(['B2', 'B3', 'B4', 'B8', 'B8A', 'B11', 'B12']) \
    .filterBounds(geometry) \
    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 5)

# Post-Fire Image Collection
img_post_collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterDate('2017-10-21', '2017-11-01') \
    .select(['B2', 'B3', 'B4', 'B8', 'B8A', 'B11', 'B12']) \
    .filterBounds(geometry) \
    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 5)

# Get dates for Pre-Fire and Post-Fire
dates_pre = img_pre_collection.aggregate_array('system:time_start').map(lambda d: ee.Date(d).format('YYYY-MM-dd')).getInfo()
dates_post = img_post_collection.aggregate_array('system:time_start').map(lambda d: ee.Date(d).format('YYYY-MM-dd')).getInfo()

print('Pre-Fire Dates:', dates_pre)
print('Post-Fire Dates:', dates_post)

# Process images (apply scaling and clip to geometry)
img_pre = img_pre_collection.mosaic().multiply(0.0001).clip(geometry)
img_post = img_post_collection.mosaic().multiply(0.0001).clip(geometry)

# Calculate NBR (Normalized Burn Ratio) for pre and post-fire images
nbr_pre = calculate_nbr(img_pre).multiply(1000).int16()
nbr_post = calculate_nbr(img_post).multiply(1000).int16()

# Visualization parameters for NBR
vis_params = {
    'min': -1000,
    'max': 1000,
    'palette': ['red', 'yellow', 'green']
}

# Create a split-panel map for Pre-Fire and Post-Fire NBR
Map = geemap.Map()

# Add Pre-Fire NBR layer
left_layer = geemap.ee_tile_layer(nbr_pre, vis_params, f'Pre-Fire NBR ({dates_pre[0] if dates_pre else "N/A"})')

# Add Post-Fire NBR layer
right_layer = geemap.ee_tile_layer(nbr_post, vis_params, f'Post-Fire NBR ({dates_post[0] if dates_post else "N/A"})')

# Set the split panel view
Map.split_map(left_layer, right_layer)

# Add legend (explaining NBR values)
legend_keys = ['Burned Areas (Low NBR)', 'Moderately Affected', 'Healthy Vegetation (High NBR)']
legend_colors = ['#FF0000', '#FFFF00', '#008000']  # red, yellow, green as HEX
Map.add_legend(title="NBR Values", keys=legend_keys, colors=legend_colors)

# Center the map on the geometry (the area of interest)
Map.centerObject(geometry, 11)

# Add text labels for title and credits
Map.add_text("Fire Impact Analysis (NBR Comparison)", fontsize=15, position='bottomright')
Map.add_text("Made with Google Earth Engine and geemap", fontsize=10, position='bottomright', offset=[0, 40])

# Display the map
Map

Pre-Fire Dates: ['2017-10-02', '2017-10-02']
Post-Fire Dates: ['2017-10-22']


Map(center=[42.268630523098274, -8.304507584460508], controls=(ZoomControl(options=['position', 'zoom_in_text'…

dNBR values are used to assess the severity of fire damage across the affected area, with thresholds separating levels from unburned to very high severity. The classified results are clipped to the region of interest and visualized using a distinct color palette, making it easy to interpret the spatial extent and intensity of the burn.

In [ ]:
# Calculate dNBR (convert to decimal values)
dnbr = nbr_pre.subtract(nbr_post).divide(1000).rename('dNBR')

# Severity classification
def classify_severity(dnbr):
    return dnbr.expression(
        "b('dNBR') <= 0.1 ? 1 : " +       # Not burned/recovered
        "(b('dNBR') <= 0.27 ? 2 : " +      # Low severity
        "(b('dNBR') <= 0.44 ? 3 : " +      # Moderate severity
        "(b('dNBR') <= 0.66 ? 4 : 5)))"    # High to very high severity
    ).rename('FireSeverity')

# Apply classification and clip
severity = classify_severity(dnbr).clip(geometry)

# Visualization settings
palette = ['#1a9850', '#91cf60', '#d9ef8b', '#fc8d59', '#d73027']
labels = [
    'Not burned/Recovered (<0.1)',
    'Low severity (0.1-0.27)',
    'Moderate severity (0.27-0.44)',
    'High severity (0.44-0.66)',
    'Very high severity (>0.66)'
]

vis_params = {
    'min': 1,
    'max': 5,
    'palette': palette,
    'opacity': 0.85
}

# Create and display the map
Map = geemap.Map(center=[42.35, -8.5], zoom=10)
Map.addLayer(severity, vis_params, 'Fire Severity (dNBR)')
Map.addLayer(geometry, {'color': 'blue'}, 'Study Area', False)
Map.add_legend(
    title="Severity Classification",
    keys=labels,
    colors=palette,
    position='bottomright'
)
Map

Map(center=[42.35, -8.5], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

dNBR values greater than 0.1 are classified into four burn severity levels, each assigned a specific color. These classified areas are masked and converted from raster to vector polygons for detailed analysis. The results are displayed on an interactive map with a Google Satellite base layer and a semi-transparent overlay representing fire severity. A custom HTML legend is included to interpret each severity class visually.

In [ ]:
import folium

# Assuming 'dnbr' is an existing Earth Engine image
dnbr_classes = dnbr \
    .where(dnbr.gt(0.1).And(dnbr.lte(0.27)), 1) \
    .where(dnbr.gt(0.27).And(dnbr.lte(0.44)), 2) \
    .where(dnbr.gt(0.44).And(dnbr.lte(0.66)), 3) \
    .where(dnbr.gt(0.66), 4)

# Mask values >= 0.10
masked_classified = dnbr_classes.updateMask(dnbr.gte(0.10))

# Convert the raster into a Feature Collection (vector polygons)
vectors = masked_classified.reduceToVectors(
    geometryType='polygon',
    reducer=ee.Reducer.countEvery(),
    scale=30,
    maxPixels=1e13
)

# Create a base Folium map centered on a location
m = folium.Map(location=[42.33, -8.41], zoom_start=10)

# Define color palette
palette = ['yellow', 'orange', 'red', 'darkred']

# Visualize the classified layer
classified_url = masked_classified.visualize(min=1, max=4, palette=palette).getMapId()
url = classified_url['tile_fetcher'].url_format

# Add Google Satellite layer as an optional layer
folium.TileLayer(
    tiles='https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
    attr='Google Satellite',
    name='Google Satellite',
    overlay=False,  # This makes it an optional base layer
    control=True  # Adds it to the layer control
).add_to(m)

# Add the dNBR classes layer (updated with color)
folium.TileLayer(
    tiles=url,
    attr='Google Earth Engine',
    name='dNBR Classes',
    opacity=0.4,
    overlay=True,  # This ensures it overlays on top of the base layer (if enabled)
    control=True  # Can also be toggled on/off from the control
).add_to(m)

# Add controls to toggle layers
folium.LayerControl().add_to(m)

# Create a custom HTML legend
legend_html = """
<div style="
    position: fixed;
    top: 10px;
    right: 10px;
    width: 220px;
    background: white;
    border: 2px solid grey;
    z-index: 1000;
    font-size: 14px;
    padding: 10px;
">
    <strong>Fire Perimeters</strong><br>
    <i style="background: yellow; width: 18px; height: 18px; float: left; margin-right: 8px; opacity: 0.7;"></i>Burned areas with low severity<br>
    <i style="background: orange; width: 18px; height: 18px; float: left; margin-right: 8px; opacity: 0.7;"></i>Burned areas with moderate-low severity<br>
    <i style="background: red; width: 18px; height: 18px; float: left; margin-right: 8px; opacity: 0.7;"></i>Burned areas with moderate-high severity<br>
    <i style="background: darkred; width: 18px; height: 18px; float: left; margin-right: 8px; opacity: 0.7;"></i>Burned areas with high severity<br>
</div>
"""

# Add the legend as a popup to the map
m.get_root().html.add_child(folium.Element(legend_html))

# Display the map
m


This code identifies isolated pixels within a classified fire severity map (dNBR). It filters out pixels with a dNBR value greater than or equal to 0.10, then uses a dilation technique to identify pixels that have no neighboring pixels. These isolated pixels are converted to vector polygons and visualized on an interactive map. The map displays the isolated pixels as semi-transparent yellow polygons, overlaid on a Google Satellite base layer, with layer controls to toggle visibility.

In [ ]:
# Filter pixels with dNBR >= 0.10 using dnbr_classes
isolated_pixels = dnbr_classes.where(dnbr_classes.gte(1), 1).updateMask(dnbr_classes.gte(1))

# Identify isolated pixels without neighbors
# Apply dilation to find neighboring pixels and then exclude them
kernel = ee.Kernel.plus(1)  # Create a 3x3 kernel for the neighborhood
dilated = isolated_pixels.focal_max(radius=1, kernel=kernel, units='pixels', iterations=1)
isolated = isolated_pixels.subtract(dilated).neq(0)  # Pixels that do not have neighbors

# Convert the raster into a Feature Collection for isolated pixels
isolated_vectors = isolated.reduceToVectors(
    geometryType='polygon',
    reducer=ee.Reducer.countEvery(),
    scale=30,
    maxPixels=1e13
)

# Visualize the interactive map with folium
m = folium.Map(location=[42.33, -8.41], zoom_start=10)

# Add vector layer of isolated pixels to the map with yellow color and transparency
isolated_vector_url = isolated_vectors.getMapId({
    'color': '#FFFF00',  # yellow
    'opacity': 0.5,      # Transparency
})
url = isolated_vector_url['tile_fetcher'].url_format

# Add Google Satellite layer as an optional layer
folium.TileLayer(
    tiles='https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
    attr='Google Satellite',
    name='Google Satellite',
    overlay=False,  # Makes it an optional base layer
    control=True  # Adds it to the layer control
).add_to(m)

# Add isolated polygons layer to the map
folium.TileLayer(
    tiles=url,
    attr='Google Earth Engine',
    name='Isolated Polygons',
    overlay=True,  # This ensures it overlays on top of the base layer (if enabled)
    control=True,  # Can be toggled on/off from the control
    opacity=0.5  # Transparency on the layer
).add_to(m)

# Add controls to toggle layers
folium.LayerControl().add_to(m)

# Display the map
m


This code identifies the 10 largest isolated pixels with a dNBR value greater than or equal to 0.10, representing fire-affected areas without neighboring pixels. These isolated pixels are converted to vector polygons, and the areas of the largest polygons are calculated in square meters and hectares. The polygons are visualized on an interactive map with distinct colors for each, and a tooltip shows the area in hectares. Additionally, Google Satellite is used as the base map layer, with controls allowing users to toggle between layers. The areas of the 10 largest polygons are printed in the console for reference.

In [ ]:
# Assuming the first part of the code has already been executed

# Get the 10 largest polygons (this part is new)
sorted_vectors = isolated_vectors.sort('count', False)
largest_10 = sorted_vectors.limit(10)

# Create an interactive map with Folium
m = folium.Map(location=[42.33, -8.41], zoom_start=10)

# Add Google Satellite layer as an optional base layer
folium.TileLayer(
    tiles='https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
    attr='Google Satellite',
    name='Google Satellite',
    overlay=False,
    control=True
).add_to(m)

# Function to get the color for each polygon
def get_color(index):
    colors = ['#FF0000', '#00FF00', '#0000FF', '#FFFF00', '#FF00FF', '#00FFFF', '#FFA500', '#800080', '#008000', '#808080']
    return colors[index % len(colors)]  # Return a color from the predefined list

# List to store the areas of the polygons
areas = []  # List to store areas in square meters and hectares

# Add the 10 largest polygons to the map and calculate the areas
for i in range(10):
    feature = ee.Feature(largest_10.toList(10).get(i))
    geom = feature.geometry().getInfo()
    area_m2 = feature.get('count').getInfo() * 30 * 30  # Area estimation in m²
    area_ha = area_m2 / 10000  # Convert area to hectares
    color = get_color(i)

    # Save the areas in the list
    areas.append((area_m2, area_ha))

    # Add each polygon to the map
    folium.GeoJson(
        geom,
        name=f'{area_ha:.2f} ha',
        style_function=lambda feature, color=color: {
            'fillColor': color,
            'color': color,
            'weight': 2,
            'fillOpacity': 0.5
        },
        highlight_function=lambda feature: {
            'weight': 3,
            'color': '#FFFFFF'
        },
        tooltip=f'Area: {area_ha:.2f} ha',
        popup=folium.Popup(f'Area: {area_ha:.2f} ha', parse_html=True)
    ).add_to(m)

# Print the areas of the 10 largest polygons
print("Areas of the 10 polygons:")
for i, (area_m2, area_ha) in enumerate(areas):
    print(f"Polygon {i+1}: Area = {area_m2} m², Area = {area_ha} ha")

# Add controls to toggle layers
folium.LayerControl().add_to(m)

# Display the map
m


Areas of the 10 polygons:
Polygon 1: Area = 96367500 m², Area = 9636.75 ha
Polygon 2: Area = 75193200 m², Area = 7519.32 ha
Polygon 3: Area = 14692500 m², Area = 1469.25 ha
Polygon 4: Area = 7473600 m², Area = 747.36 ha
Polygon 5: Area = 7189200 m², Area = 718.92 ha
Polygon 6: Area = 5152500 m², Area = 515.25 ha
Polygon 7: Area = 2835000 m², Area = 283.5 ha
Polygon 8: Area = 2122200 m², Area = 212.22 ha
Polygon 9: Area = 1216800 m², Area = 121.68 ha
Polygon 10: Area = 985500 m², Area = 98.55 ha


The code filters isolated pixels with dNBR values ≥ 0.10, applies dilation to identify isolated pixels without neighbors, and reduces them to vector polygons. It then sorts the polygons by area, allowing the user to select a polygon by specifying its area. After smoothing the polygon’s coordinates, it exports the selected polygon as a GeoJSON file, saving it in the current working directory with a filename that includes the polygon's index and area.

In [ ]:
import numpy as np
import os

# Function to smooth polygon coordinates using iterative averaging
def smooth_polygon(coords, iterations=5, alpha=0.5):
    smoothed_coords = np.array(coords)
    for _ in range(iterations):
        new_coords = smoothed_coords.copy()
        for i in range(1, len(smoothed_coords) - 1):
            new_coords[i] = alpha * smoothed_coords[i] + (1 - alpha) * (smoothed_coords[i - 1] + smoothed_coords[i + 1]) / 2
        smoothed_coords = new_coords
    return smoothed_coords.tolist()

# Output directory for local download
out_dir = os.getcwd()
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

# Filter pixels with dNBR >= 0.10
isolated_pixels = dnbr_classes.where(dnbr_classes.gte(1), 1).updateMask(dnbr_classes.gte(1))

# Identify isolated pixels without neighbors
kernel = ee.Kernel.plus(1)
dilated = isolated_pixels.focal_max(radius=1, kernel=kernel, units='pixels', iterations=1)
isolated = isolated_pixels.subtract(dilated).neq(0)

# Convert raster to vector
isolated_vectors = isolated.reduceToVectors(
    geometryType='polygon',
    reducer=ee.Reducer.countEvery(),
    scale=30,
    maxPixels=1e13
)

# Sort and select top 10 polygons
largest_10 = isolated_vectors.sort('count', False).limit(10)

# Ask user for area
selected_area = float(input("Enter the area of the polygon you want to export (in ha):"))

# Find matching polygon by area
def find_polygon_by_area(area):
    for index, feature in enumerate(largest_10.getInfo()['features']):
        computed_area = feature['properties']['count'] * 30 * 30 / 10000
        if abs(computed_area - area) < 0.01:
            return index
    return None

# Export if match is found
selected_index = find_polygon_by_area(selected_area)

if selected_index is not None:
    print(f"Exporting the polygon with an area of {selected_area} ha (Polygon {selected_index + 1}).")

    # Get the polygon feature
    selected_feature = ee.Feature(largest_10.toList(10).get(selected_index))
    area_m2 = selected_feature.get('count').getInfo() * 30 * 30
    area_ha = area_m2 / 10000

    # Extract coordinates directly (no buffer)
    geometry = selected_feature.geometry().getInfo()
    coords = geometry['coordinates'][0]

    # Smooth polygon coordinates
    smoothed_coords = smooth_polygon(coords, iterations=5, alpha=0.5)

    # Ensure the polygon is closed
    if smoothed_coords[0] != smoothed_coords[-1]:
        smoothed_coords.append(smoothed_coords[0])

    # Create smoothed polygon geometry
    smoothed_polygon = ee.Geometry.Polygon([smoothed_coords])
    selected_fc = ee.FeatureCollection([ee.Feature(smoothed_polygon, {'area_ha': area_ha})])

# Set output path and filename with .geojson extension
filename = os.path.join(out_dir, f"polygon_{selected_index+1}_area_{int(area_ha)}ha.geojson")

# Export the FeatureCollection as GeoJSON
geemap.ee_export_vector(
    ee_object=selected_fc,
    filename=filename,
    verbose=True
)

print(f"GeoJSON file successfully downloaded to: {filename}")

Enter the area of the polygon you want to export (in ha):283.5
Exporting the polygon with an area of 283.5 ha (Polygon 7).
Generating URL ...
Please wait ...
Data downloaded to /content/polygon_7_area_283ha.geojson
GeoJSON file successfully downloaded to: /content/polygon_7_area_283ha.geojson
